In [8]:
import pandas as pd
m = pd.read_csv('movie.csv')
r = pd.read_csv('rating.csv')

m.describe(include=['object'])

m['movieId'].info

m.columns

r.info

df = pd.merge(m, r, on='movieId', how='inner')

m.head(3)

r.head(3)

df = pd.merge(m, r, on='movieId', how='inner')

df.info()

df.drop(['title' ], axis=1, inplace = True)

df.head()

df['freq'] = df['userId'].map(df['userId'].value_counts())

df.shape

df_5 = df[df['freq'] >= 100].drop(columns=['freq'])

df_5.shape

p = df_5.pivot( index ='userId', columns='movieId', values = "rating").reset_index(drop=True)

p.head()

p.shape

p = p.iloc[0:5000,0:5000]

p.shape

p.fillna(0, inplace=True)







from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation
dis = 1- pairwise_distances(p.values,metric = "cosine")
disk = pd.DataFrame(dis)

user_ids = r.userId.unique()

# Determine the minimum size to ensure both dist and user_ids match in length
min_size = min(len(user_ids), disk.shape[0])

# Truncate dist and user_ids to match dimensions
disk = disk.iloc[:min_size, :min_size]
disk.index = user_ids[:min_size]
disk.columns = user_ids[:min_size]




disk.iloc[0:5,0:5]

disk.shape

import numpy as np

# Use numpy's fill_diagonal function directly on the underlying numpy array
np.fill_diagonal(disk.values, 0)

disk.iloc[0:5,0:5]

disk.idxmax(axis=1)[0:5]

def get_same(user1,user2) : 
    com_movies = r[r.userId == user1].merge(r[r.userId == user2], on = "movieId", how = 'inner')
    return com_movies.merge(m, on = 'movieId')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   title      object 
 2   genres     object 
 3   userId     int64  
 4   rating     float64
 5   timestamp  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 915.5+ MB


In [10]:
def recommend_movies(user_id, disk, ratings_df, movies_df, top_n=5):
    # Step 1: Find the most similar users to the input user_id
    similar_users = disk.loc[user_id].nlargest(10).index  # Top 10 similar users

    # Step 2: Get movies watched by similar users that the input user hasn't seen
    user_movies = set(ratings_df[ratings_df.userId == user_id].movieId)
    recommendations = {}

    for sim_user in similar_users:
        sim_user_movies = ratings_df[ratings_df.userId == sim_user].movieId
        for movie in sim_user_movies:
            if movie not in user_movies:  # Only recommend movies not watched by input user
                recommendations[movie] = recommendations.get(movie, 0) + 1

    # Step 3: Rank recommendations by frequency and return the top_n
    recommended_movies = sorted(recommendations, key=recommendations.get, reverse=True)[:top_n]
    
    # Get movie titles for the recommendations
    recommended_movie_titles = movies_df[movies_df.movieId.isin(recommended_movies)]
    return recommended_movie_titles[['movieId', 'title']]

In [32]:
# Get recommendations for a specific user
recommendations = recommend_movies(user_id=1234, disk=disk, ratings_df=r, movies_df=m, top_n=10)
print(recommendations)


KeyError: 9067